In [14]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [15]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (5000, 28, 28) (5000,)
Test set (5000, 28, 28) (5000,)


In [16]:
image_size = 28
num_labels = 5

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 784) (100000, 5)
Validation set (5000, 784) (5000, 5)
Test set (5000, 784) (5000, 5)


In [17]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [22]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  loss = loss + 0.003 * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [21]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 15.8456
Minibatch accuracy: 18.0%
Validation accuracy: 29.4%
Minibatch loss at step 500 : 1.34076
Minibatch accuracy: 85.2%
Validation accuracy: 82.4%
Minibatch loss at step 1000 : 0.679821
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 1500 : 0.409229
Minibatch accuracy: 84.4%
Validation accuracy: 85.3%
Minibatch loss at step 2000 : 0.500548
Minibatch accuracy: 84.4%
Validation accuracy: 85.2%
Minibatch loss at step 2500 : 0.593961
Minibatch accuracy: 82.0%
Validation accuracy: 85.2%
Minibatch loss at step 3000 : 0.371981
Minibatch accuracy: 87.5%
Validation accuracy: 85.4%
Test accuracy: 92.2%


In [23]:
batch_size = 128
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Set hidden layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)

  # Set final layer variables
  weights = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))

  biases = tf.Variable(tf.zeros([num_labels]))
    
  logits = tf.matmul(hidden1, weights) + biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  ## validation operation
  valid_prediction1 =  tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_prediction1, weights) + biases)
  

  #test operation
  
  test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)

In [24]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 652.89
Minibatch accuracy: 28.9%
Validation accuracy: 36.8%
Minibatch loss at step 500 : 224.198
Minibatch accuracy: 87.5%
Validation accuracy: 83.8%
Minibatch loss at step 1000 : 103.525
Minibatch accuracy: 89.8%
Validation accuracy: 87.7%
Minibatch loss at step 1500 : 48.6562
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 2000 : 23.1488
Minibatch accuracy: 93.8%
Validation accuracy: 89.8%
Minibatch loss at step 2500 : 11.1437
Minibatch accuracy: 88.3%
Validation accuracy: 90.5%
Minibatch loss at step 3000 : 5.34842
Minibatch accuracy: 95.3%
Validation accuracy: 90.8%
Test accuracy: 96.1%


In [35]:
batch_size = 128
hidden1_units = 1024
beta = 0.003

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Set hidden layer 1
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden1_units]))
  biases1 = tf.Variable(tf.zeros([hidden1_units]))
    
  hidden1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  hidden1=tf.nn.dropout(hidden1,0.6)

  # Set final layer variables
  weights = tf.Variable(
    tf.truncated_normal([hidden1_units, num_labels]))

  biases = tf.Variable(tf.zeros([num_labels]))
    
  logits = tf.matmul(hidden1, weights) + biases
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  loss = loss + 0.5*beta*(tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.04, global_step,1000,0.90,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  ## validation operation
  valid_prediction1 =  tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_prediction1, weights) + biases)
  
  test_prediction1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  test_prediction = tf.nn.softmax(tf.matmul(test_prediction1, weights) + biases)

In [36]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step", step, ":", l)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0 : 817.713
Minibatch accuracy: 21.9%
Validation accuracy: 40.2%
Test accuracy: 43.4%
Minibatch loss at step 500 : 466.398
Minibatch accuracy: 82.8%
Validation accuracy: 86.5%
Test accuracy: 93.4%
Minibatch loss at step 1000 : 429.566
Minibatch accuracy: 81.2%
Validation accuracy: 86.9%
Test accuracy: 93.5%
Minibatch loss at step 1500 : 397.752
Minibatch accuracy: 86.7%
Validation accuracy: 87.8%
Test accuracy: 94.4%
Minibatch loss at step 2000 : 374.503
Minibatch accuracy: 84.4%
Validation accuracy: 87.7%
Test accuracy: 94.3%
Minibatch loss at step 2500 : 359.211
Minibatch accuracy: 76.6%
Validation accuracy: 87.1%
Test accuracy: 93.9%
Minibatch loss at step 3000 : 337.748
Minibatch accuracy: 81.2%
Validation accuracy: 88.0%
Test accuracy: 94.6%
Minibatch loss at step 3500 : 321.189
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Test accuracy: 93.8%
Minibatch loss at step

In [28]:

hidden_size=1024
hidden2_size=300
batch_size=128
beta1=0.01
beta2=0.01
beta3=0.0001

graph=tf.Graph()
with graph.as_default():
    tf_train_dataset=tf.placeholder(np.float32,shape=(batch_size,image_size*image_size))
    tf_train_labels=tf.placeholder(np.float32,shape=(batch_size,num_labels))
    tf_valid_dataset=tf.constant(valid_dataset)
    tf_test_dataset=tf.constant(test_dataset)
    
    weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    
    hidden_1=tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
    
#     hidden_1=tf.nn.dropout(hidden_1,0.5)
    
    weights_2 = tf.Variable(
    tf.truncated_normal([hidden_size, hidden2_size]))
    biases_2 = tf.Variable(tf.zeros([hidden2_size]))
    
    hidden_2=tf.nn.relu(tf.matmul(hidden_1,weights_2)+biases_2)
    
#     hidden_2=tf.nn.dropout(hidden_2,0.5)
  
    weights_3 = tf.Variable(
    tf.truncated_normal([hidden2_size, num_labels]))
    biases_3 = tf.Variable(tf.zeros([num_labels]))
    
    logits=tf.matmul(hidden_2,weights_3)+biases_3
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+beta1*tf.nn.l2_loss(weights_1)+beta2*tf.nn.l2_loss(weights_2)
    +beta3*tf.nn.l2_loss(weights_3)

    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.04, global_step,1000,0.90,staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
#     optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1),weights_2)+biases_2),weights_3)
    +biases_3)
    test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1),weights_2)+biases_2),weights_3)
    +biases_3)
    

In [29]:

num_steps=9001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('initialized')
    for step in range(num_steps):
        offset=(step*batch_size)%(train_dataset.shape[0]-batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        
        _,l,predictions=session.run([optimizer,loss,train_prediction],feed_dict=feed_dict)
        
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
            
        global_step+=1
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
initialized
Minibatch loss at step 0: 6399.591797
Minibatch accuracy: 23.4%
Validation accuracy: 20.9%
Minibatch loss at step 1000: 1957.444092
Minibatch accuracy: 67.2%
Validation accuracy: 72.0%
Minibatch loss at step 2000: 952.731201
Minibatch accuracy: 84.4%
Validation accuracy: 79.2%
Minibatch loss at step 3000: 498.470184
Minibatch accuracy: 79.7%
Validation accuracy: 82.3%
Minibatch loss at step 4000: 278.381378
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 5000: 164.767166
Minibatch accuracy: 87.5%
Validation accuracy: 84.9%
Minibatch loss at step 6000: 102.834030
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 7000: 67.317741
Minibatch accuracy: 89.8%
Validation accuracy: 86.9%
Minibatch loss at step 8000: 46.047302
Minibatch accuracy: 89.8%
Validation accuracy: 87.7%
Minibatch loss at step 9000: 32.599056
Minibatch accuracy: 95.3%
Validation accu